In [5]:
import random
import string
import re
import numpy as np
import tensorflow as tf
import keras
from tensorflow.keras.layers import MultiHeadAttention

In [6]:
with open('ukr.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')[:-1]

print("Data from ukr.txt")
for _ in range(3):
    print(random.choice(lines))

text_pairs = []
for line in lines:
    eng, ukr, _ = line.split('\t')
    ukr = '[s] ' + ukr + ' [e]'
    text_pairs.append((eng, ukr))

print("\nSample from text_pairs")
for t in range(5):
    print(random.choice(text_pairs))

random.shuffle(text_pairs)
text_pairs = text_pairs[:50000]
num_val = int(0.15 * len(text_pairs))
num_train = len(text_pairs) - 2 * num_val
train_pairs = text_pairs[:num_train]
val_pairs = text_pairs[num_train: num_train + num_val]
test_pairs = text_pairs[num_train + num_val:]

Data from ukr.txt
Please take some.	Будь ласка, візьміть декілька.	CC-BY 2.0 (France) Attribution: tatoeba.org #2854251 (CK) & #495726 (ChickenKiev)
I forgot his address.	Я забув його адресу.	CC-BY 2.0 (France) Attribution: tatoeba.org #260664 (CK) & #6667474 (deniko)
Didn't you know that Tom lived in Australia when he was a kid?	Хіба ви не знали, що Том у дитинстві жив у Австралії?	CC-BY 2.0 (France) Attribution: tatoeba.org #7194483 (CK) & #7429873 (deniko)

Sample from text_pairs
('What do you think Tom will do?', '[s] Що, на твою думку, робитиме Том? [e]')
('Mind your own business!', '[s] Не твоя справа! [e]')
('Science is very exciting.', '[s] Наука дуже захоплююча. [e]')
('This diet is full of vitamins.', '[s] Ця дієта повна вітамінів. [e]')
('He is about my age.', '[s] Він зі мною приблизно одного віку. [e]')


In [7]:
strip_chars = string.punctuation.replace('[', '')
strip_chars = strip_chars.replace(']', '')

vocabulary_size = 15000
sequence_length = 20
batch_size = 64

def ukr_standardization(input_string):
    return tf.strings.regex_replace(tf.strings.lower(input_string), '[%s]' % re.escape(strip_chars), '')

eng_vector = keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    output_mode='int',
    output_sequence_length=sequence_length,
)
ukr_vector = keras.layers.TextVectorization(
    max_tokens=vocabulary_size,
    output_mode='int',
    output_sequence_length=sequence_length + 1,
    standardize=ukr_standardization,
)
train_eng = [pair[0] for pair in train_pairs]
train_ukr = [pair[1] for pair in train_pairs]
eng_vector.adapt(train_eng)
ukr_vector.adapt(train_ukr)

for v in [eng_vector, ukr_vector]:
    print(len(v.get_vocabulary()))
    print(v.get_vocabulary()[10:20])

6456
[np.str_('do'), np.str_('im'), np.str_('he'), np.str_('dont'), np.str_('was'), np.str_('have'), np.str_('mary'), np.str_('me'), np.str_('in'), np.str_('it')]
15000
[np.str_('на'), np.str_('Мері'), np.str_('ти'), np.str_('Тома'), np.str_('з'), np.str_('я'), np.str_('у'), np.str_('Це'), np.str_('У'), np.str_('мене')]


In [8]:
def format_dataset(eng, ukr):
    eng = eng_vector(eng)
    ukr = ukr_vector(ukr)
    return ({"encoder_inputs": eng, "decoder_inputs": ukr[:, :-1]}, ukr[:, 1:])

def make_dataset(pairs):
    eng_texts, ukr_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    ukr_texts = list(ukr_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ukr_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.cache().shuffle(2048).prefetch(16)

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [8]:
for inputs, targets in train_ds.take(1):
    print(f'encoder inputs shape: {inputs["encoder_inputs"].shape}')
    print(f'decoder inputs shape: {inputs["decoder_inputs"].shape}')
    print(f"targets shape: {targets.shape}")

encoder inputs shape: (64, 20)
decoder inputs shape: (64, 20)
targets shape: (64, 20)


2024-12-27 15:38:40.758878: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
# Генерація позиційного кодування
def get_positional_encoding(seq_len, embed_dim):
    position = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
    div_term = tf.exp(tf.range(0, embed_dim, 2, dtype=tf.float32) * -(tf.math.log(10000.0) / embed_dim))
    
    sinusoidal = tf.concat([tf.sin(position * div_term), tf.cos(position * div_term)], axis=-1)
    return sinusoidal

# Оновлення класів TransformerEncoder та TransformerDecoder з додаванням позиційного кодування
class TransformerEncoder(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, latent_dim):
        super(TransformerEncoder, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.latent_dim = latent_dim
        self.attention = MultiHeadAttention(num_heads, embed_dim)  # Assuming you have this implemented

    def call(self, inputs, training=False):
        seq_len = tf.shape(inputs)[1]
        pos_encoding = get_positional_encoding(seq_len, self.embed_dim)
        inputs = inputs + pos_encoding  # Add positional encoding to the input
        attn_output = self.attention(inputs, inputs)  # Example attention mechanism
        return attn_output

class TransformerDecoder(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerDecoder, self).__init__()
        self.attention1 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.attention2 = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = tf.keras.Sequential([keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim)])
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)
        self.dropout3 = keras.layers.Dropout(rate)

    def call(self, inputs, enc_output, training=False):
        seq_len = tf.shape(inputs)[1]
        pos_encoding = get_positional_encoding(seq_len, inputs.shape[-1])
        inputs = inputs + pos_encoding
        attn_output1 = self.attention1(inputs, inputs)
        attn_output1 = self.dropout1(attn_output1, training=training)
        out1 = self.layernorm1(inputs + attn_output1)
        attn_output2 = self.attention2(out1, enc_output)
        attn_output2 = self.dropout2(attn_output2, training=training)
        out2 = self.layernorm2(out1 + attn_output2)
        ffn_output = self.dense_proj(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        return self.layernorm3(out2 + ffn_output)

In [12]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.layers.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, num_heads, latent_dim)(x, training=True)

decoder_inputs = keras.layers.Input(shape=(None,), dtype="int64", name="decoder_inputs")
x = keras.layers.Embedding(input_dim=vocabulary_size, output_dim=embed_dim)(decoder_inputs)

x = TransformerDecoder(embed_dim, num_heads, latent_dim)(x, encoder_outputs, training=True)
decoder_outputs = keras.layers.Dense(vocabulary_size, activation="softmax")(x)

transformer = keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs, name="transformer")

In [13]:
epochs = 30

transformer.compile("rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Epoch 1/30


I0000 00:00:1735313922.917608  123740 service.cc:148] XLA service 0x7f0274007190 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1735313922.917638  123740 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 SUPER, Compute Capability 8.9
2024-12-27 15:38:42.969843: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1735313923.034837  123740 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
I0000 00:00:1735313923.354426  123740 cuda_dnn.cc:529] Loaded cuDNN version 90300
2024-12-27 15:38:44.481460: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_21', 108 bytes spill stores, 108 bytes spill loads

20

  6/547 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.4184 - loss: 8.2801       

I0000 00:00:1735313935.958902  123740 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


400/547 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.7171 - loss: 2.7455 

W0000 00:00:1735313945.064447  123742 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-12-27 15:39:05.987116: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_74', 8 bytes spill stores, 8 bytes spill loads

2024-12-27 15:39:06.353835: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_19', 236 bytes spill stores, 236 bytes spill loads

2024-12-27 15:39:06.501497: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_67', 144 bytes spill stores, 296 bytes spill loads

2024-12-27 15:39:06.743717: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas war

545/547 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - accuracy: 0.7255 - loss: 2.5486

W0000 00:00:1735313960.742205  123741 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
W0000 00:00:1735313961.413402  123741 assert_op.cc:38] Ignoring Assert operator compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert
2024-12-27 15:39:22.164991: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_22_0', 352 bytes spill stores, 216 bytes spill loads

2024-12-27 15:39:22.349760: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_25', 8 bytes spill stores, 8 bytes spill loads

2024-12-27 15:39:22.367401: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memor

547/547 ━━━━━━━━━━━━━━━━━━━━ 43s 51ms/step - accuracy: 0.7256 - loss: 2.5453 - val_accuracy: 0.7778 - val_loss: 1.5418
Epoch 2/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.7797 - loss: 1.5600 - val_accuracy: 0.7860 - val_loss: 1.3889
Epoch 3/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.8089 - loss: 1.3387 - val_accuracy: 0.8755 - val_loss: 0.8471
Epoch 4/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.8769 - loss: 0.8766 - val_accuracy: 0.9097 - val_loss: 0.6178
Epoch 5/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.8804 - loss: 1.0008 - val_accuracy: 0.9324 - val_loss: 0.4573
Epoch 6/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.9246 - loss: 0.5187 - val_accuracy: 0.9471 - val_loss: 0.3571
Epoch 7/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.9374 - loss: 0.4178 - val_accuracy: 0.9533 - val_loss: 0.3089
Epoch 8/30
547/547 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.9467 - loss: 0.3466 - val_accurac

In [19]:
ukr_vocab = ukr_vector.get_vocabulary()
ukr_index_lookup = dict(zip(range(len(ukr_vocab)), ukr_vocab))
max_decoded_sentence_length = 20  

def decode_sequence(input_sentence):
    # Токенізація
    tokenized_input_sentence = eng_vector([input_sentence])
    
    decoded_sentence = "[s]" 
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = ukr_vector([decoded_sentence])[:, :-1]
        
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])  
        sampled_token = ukr_index_lookup[sampled_token_index]  
        
        decoded_sentence += " " + sampled_token 
        
        if sampled_token == "[e]":
            break
    
    return decoded_sentence


test_eng_texts = [pair[0] for pair in test_pairs]

print(f'Результат:\n{"-"*50}')
for _ in range(10):
    input_sentence = random.choice(test_eng_texts)  
    translated = decode_sequence(input_sentence)  
    print(f'INPUT: {input_sentence}\nOUTPUT: {translated}\n{"-"*50}')

Результат:
--------------------------------------------------
INPUT: These aren't for you.
OUTPUT: [s] Знаю пройтися згнили гасло таборі ведмедика здійснилася власнику будьласка будьласка пропав вмятина вмятина вмятина свідоцтво столиці в офісі [e]
--------------------------------------------------
INPUT: We found Tom.
OUTPUT: [s] від Перестань будівлі компютерами таборі ведмедика ведмедика ведмедика будьласка будьласка пропав вмятина вмятина вмятина свідоцтво в в в офісі в
--------------------------------------------------
INPUT: I'll live on welfare.
OUTPUT: [s] Знаю пляж промова промова таборі ведмедика ведмедика власнику будьласка пропав хмари вмятина вмятина вмятина свідоцтво столиці в офісі [e]
--------------------------------------------------
INPUT: We love you.
OUTPUT: [s] Знаю скорочується згнили молодик платформу платформу кастрюлі персонажів будьласка будьласка пропав вмятина вмятина вмятина пошті в в в офісі в
--------------------------------------------------
INPUT: We lo